# Example output plots <a class="anchor" id="top"></a>


### Example run
The example data used to make these plots is currently in the same directory as the Jupyter notebook - you will want to change this to where you have saved your results.
The run shown here is for a 500km planetesimal, accreted 0.8Ma after CAI formation, $X_{S,0}$=26.7 wt%, all other parameters are given by run 1 in auto_params.csv. To make plots for a specific run, **you only need to change the information in the second code cell**. Everything else is automatic. Figure numbers refer to plots in Sanderson et. al. 2024a "Unlocking planetesimal magnetic field histories: a refined, versatile model for thermal evolution and dynamo generation" submitted to Icarus.

+ [Summary stats](#stats)
+ [Temperature profile - Figure 6](#temp)
+ [Heat fluxes - Figure 7](#flux)
+ [Field strength and Reynolds number - Figure 8](#Bfield)

### Set-up

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import sys
sys.path.append('../')
from load_info import load_run_info, load_run_results

#scale time to Myr
from plot_params import Myr, f0

Choose run for main thermal plots and where to save files. 

**Change the parameters in this box**

In [ ]:
run=1 #run you want to plot
save = False # do you want to save your figures?
automated = False #did you calculate runs individually or automated
log_time = True #do you want to plot time logarithmically
path = '' #path to files 
save_path = '../Plots/' #path to where you save your files

Load results for differentiation

In [ ]:
npzfile = np.load(f'{path}run_{run}_diff.npz')
Tdiff = npzfile['Tdiff'] #temperature profile [K]
iron = npzfile['Xfe'] #Fe-FeS melt fraction
silicate_d = npzfile['Xsi'] #silicate melt fraction
tdiff = npzfile['t_diff'] #time points [s]
Ra_d = npzfile['Ra'] #Rayleigh number
Ra_crit_d = npzfile['Ra_crit'] #critical Rayleigh number
d0_diff = npzfile['d0'] #stagnant lid thickness [m]

Load results for thermal evolution

In [ ]:
npzfile = np.load(f'{path}run_{run}.npz')
Tc= npzfile['Tc'] #central core temperature [K]
Tc_conv = npzfile['Tc_conv'] #convective core temperature [K], 0 when the core isn't convecting
Tcmb = npzfile['Tcmb'] #CMB temperature [K]
Tm_mid = npzfile['Tm_mid'] # mid-mantle temperature [K]
Tm_conv = npzfile['Tm_conv'] #convective mantle temperature [K], 0 when the mantle isn't convecting 
Tm_surf = npzfile['Tm_surf'] # temperature one cell below the surface [K]
T_profile = npzfile['T_profile'] #full temperature profile, first dimension is time, second is radius T_profile[:,-1] is surface
f = npzfile['f'] #fractional inner core radius

t = npzfile['t'] #time in s
Rem = npzfile['Rem'] # magnetic Reynolds number 
B = npzfile['B']/1e-6 # magnetic field strength [T] 
buoyr = npzfile['buoyr'] #compositional buoyancy flux and thermal buoyancy flux
Flux = npzfile['Flux'] #array of fluxes
Ra = npzfile['Ra'] # model Rayleigh number (whichever RaH or RanoH is greater)
RaH = npzfile['RaH'] #radiogenic heating Rayleigh number
RanoH = npzfile['RanoH'] # no-internal heating Rayleigh number
Racrit = npzfile['Racrit'] #critical Rayleigh number 
eta = npzfile['eta'] #mantle viscosity [Pas]
d0 = npzfile['d0'] #stagnant lid thickness [m]
min_unstable = npzfile['min_unstable'] #
Urey = npzfile['Ur'] #Urey ratio
Xs = npzfile['Xs'] #liquid core sulfur content [wt %]
dl = npzfile['dl'] #CMB boundary layer thickness in mantle [m]
dc = npzfile['dc'] #CMB boundary layer thickness in core [m]
Fs = Flux[0] #surface heat flux [W/m^2]
Fcmb = Flux[1] #CMB heat flux [W/m^2]
Fad = Flux[2] #adiabatic heat flux [W/m^2]
Frad = Flux[3] #radiogenic heat flux [W/m^2]
qcore = npzfile['qcore']
#core heat sources
qcore = qcore/1e9 #convert to GW
qrad = qcore[0,:] #radiogenic heat [GW]
qs = qcore[1,:] #secular cooling [GW]
ql = qcore[2,:] #latent heat [GW]
qg = qcore[3,:] #release of GPE [GW], zero unless you choose to turn it on

Concatenate shared variables.

After this step the first dimension of Tall is radius and the second is time. For example, `Tall[-1,:]` is the surface temperature at all times.

In [ ]:
Tall = np.transpose(np.vstack((Tdiff,T_profile))) #temperature profile [K]
tall = np.append(tdiff,t) #time [s]
Ra_all = np.append(Ra_d,Ra) #Rayleigh number
Ra_crit_all = np.append(Ra_crit_d,Racrit) #critical Rayleigh number
d0_all = np.append(d0_diff,d0) #stagnant lid thickness

Scale time by Myr

In [ ]:
t_plot_all = tall/Myr
t_plot_t = t/Myr

Run info

r = planetesimal radius, rcr = core radius fraction, dr = grid spacing, tstart = accretion time, dt = timestep, viscosity = viscosity model (Sanderson et. al. or Dodds, Bryson), icfrac = core solidification endmember

In [ ]:
if automated == True:
    r, rcr, dr, tstart, dt, viscosity, icfrac = load_run_info(run,f'{path}auto_params.csv')
else: 
    r, rcr, dr, tstart, dt, viscosity, icfrac = load_run_info(run,f'{path}run_info.csv')

## Summary statistics <a class="anchor" id="stats"></a>
<p align="right">(<a href="#top">back to top</a>)</p>

Load in results data

In [ ]:
results = load_run_results(run,f'{path}run_results.csv')

Load in run parameters

In [ ]:
if automated == True:
    params = pd.read_csv(f'{path}auto_params.csv',skiprows=[1])
    params = params[params['run']==run]
else:
    params = pd.read_csv(f'{path}run_info.csv',skiprows=[1])
    params = params[params['run']==run]

Assign to variables values that will be used later. See METADATA.md for other values that could be imported from `results.csv`

In [ ]:
n_cells = int(r/dr) #number of cells needed to span the body
nccells = round((n_cells-3)*(rcr))+2 #number of cells needed to span core (inc. centre and CMB)
nmantle = n_cells - nccells +1 #number of cells needed to span mantle plus one extra for CMB
rc = (nccells-1)*dr #radius of core [m], subtract one for centre
terode = results.at[0,'terode'] #removal of all core thermal stratification [Myr]
tstrat_remove = results.at[0,'tstrat_remove'] #beginning of erosion of core thermal stratification [Myr]
diff_time = results.at[0,'diff_time'] #time of differentiation [Myr]
peak_coreT = np.amax(Tall[:,:nmantle]) #peak core temperature [K]
loc_max = np.where(Tall[:,:nmantle]==peak_coreT)[1][0] #take the set of time coordinates and first value (they should all be the same)
tcoremax = tall[loc_max]/Myr #time of peak core temperature [Myr]
fcond_t = results.at[0,'fcond_t'] # time of cessation of convection [Myr]
diff_T =results.at[0,'diff_T'] #differentiation temperature [K]
tsolid_start =results.at[0,'tsolid_start'] #onset of core solidification [Myr]
#dynamo on and off times
var_results = pd.read_csv(path+'run_results.csv',skiprows=[1])
on1=var_results.loc[var_results['run']==run,'magon_1'].values[0]
off1=var_results.loc[var_results['run']==run,'magoff_1'].values[0]
on2=var_results.loc[var_results['run']==run,'magon_2'].values[0]
off2=var_results.loc[var_results['run']==run,'magoff_2'].values[0]
on3=var_results.loc[var_results['run']==run,'magon_3'].values[0]
off3=var_results.loc[var_results['run']==run,'magoff_3'].values[0]

Print a summary

In [ ]:
print(f"Differentiation is at {results.at[0,'diff_time']:.2f} Myr")
print(f"The temperature at differentiation is at {results.at[0,'diff_T']:.2f}K")
print(f"Peak magma ocean temp is {results.at[0,'peakT']:.0f}K at {results.at[0,'tmax']:.2f} Myr")
print(f"Stratification starts at {results.at[0,'tstrat_start']:.2f} Myr")
print(f"Mantle hotter than the core until {results.at[0,'tstrat_remove']:.2f} Myr")
print(f"Erosion of stratification by {results.at[0,'terode']:.2f} Myr") 
print(f"End of mantle convection by {results.at[0,'fcond_t']:.2f} Myr")
print(f"The maximum thermal Rem is {results.at[0,'max_Rtherm']:.2f} at {results.at[0,'max_Rtherm']:.2f} Myr")
print(f"The maximum thermal field strength is {results.at[0,'max_Btherm']:.2e}T at {results.at[0,'max_Bthermt']:.2f} Myr")
print(f"The maximum compositional Rem is {results.at[0,'max_Rcomp']:.2f}")
print(f"The maximum compositional field strength is {results.at[0,'max_Bcomp']:.2e}")
print(f'The dynamo starts at {on1:.2f} Myr, stops at {off1:.2f} Myr and lasts {off1-on1:.2f} Myr')
if on2 > 0:
    print(f'The second dynamo starts at {on2:.2f} Myr, stops at {off2:.2f} Myr and lasts {off2-on2:.2f} Myr')
if on3 > 0:
    print(f'The third dynamo starts at {on3:.2f} Myr, stops at {off3:.2f} Myr and lasts {off3-on3:.2f} Myr')
print(f"Core solidification begins at {results.at[0,'tsolid_start']:.2f} Ma and ends at {results.at[0,'tsolid']:.2f} Ma")

# Example run

## Temperature profile <a class="anchor" id="temp"></a>
<p align="right">(<a href="#top">back to top</a>)</p>

Create r array for plotting and prelog time to speed up plotting

In [ ]:
rplot = np.arange(0,int(r)+dr,int(dr))/1e3
r_unstable=np.array([]) 
for ind in min_unstable:
    r_unstable = np.append(r_unstable,rplot[int(ind)])


#log time
if log_time == True:
    tpt = np.log10(t_plot_t)
    tpa = np.log10(t_plot_all)
    lfcond = np.log10(fcond_t)
    on1l = np.log10(on1)
    on2l = np.log10(on2)
    on3l = np.log10(on3)
    off1l = np.log10(off1)
    off2l = np.log10(off3)
    off3l = np.log10(off3)
else: 
    tpt = t_plot_t
    tpa = t_plot_all
    lfcond = fcond_t
    on1l = on1
    on2l = on2
    on3l = on3
    off1l = off1
    off2l = off2
    off3l = off3

Make figure - Figure 6 in Sanderson et. al. 2024a

In [ ]:
plt.figure(figsize=[10,20/3])
plt.pcolormesh(tpa[::2],rplot[::2],np.transpose(Tall[::2,::2]),shading = 'gouraud',vmin=200,vmax=1600) 
plt.hlines(rc/1e3,tpt[0],max(tpa),linestyle='--',color='black',label='CMB')
plt.vlines(tpt[0],0,r/1e3,linestyle='-.',label='Differentiation')
plt.annotate(text='', xy=(on1l,rc/1e4), xytext=(off1l,rc/1e4), arrowprops=dict(arrowstyle='<->'))
plt.plot(tpt,r_unstable,linestyle='dotted',label='Convecting core')
if on2 > 0:
    plt.annotate(text='', xy=(on2l,rc/1e4), xytext=(off2l,rc/1e4), arrowprops=dict(arrowstyle='<->'))
if on3 > 0: 
    plt.annotate(text='', xy=(on3l,rc/1e4), xytext=(off3l,rc/1e4), arrowprops=dict(arrowstyle='<->'))


if np.any(t_plot_t<fcond_t):
    plt.plot(tpt[(t_plot_t<=fcond_t)&(d0<(r-rc))],(r-d0[(t_plot_t<=fcond_t)&(d0<(r-rc))])/1e3,linestyle='dashed',label='base of $\delta_0$',color='blue')
    plt.plot(tpt[(t_plot_t<=fcond_t)&(r_unstable==0)][1:],(rc+dl[(t_plot_t<=fcond_t)&(r_unstable==0)][1:])/1e3,linestyle='dotted',label='top of $\delta_l$',color='blue')
    plt.vlines(tpt[t_plot_t<=fcond_t][-1],r/1e3,rc/1e3,linestyle='dotted',label='conductive mantle',color='red')
plt.plot(tpt[f<f0],f[f<f0]*rc/1e3,linestyle='-.',color='black',label='Top of liquid core')
#labels and limits
plt.ylabel('Distance from centre of planetesimal/km')
if log_time == False:
    plt.xlabel('Time / Ma')
else:
    plt.xlabel('log10(Time / Ma)')
plt.colorbar(label='Temperature/K')
plt.ylim(bottom=0)
plt.legend(bbox_to_anchor=[1.6,0.5])
#plt.xscale('log')
if save == True:
    plt.savefig(f'{save_path}run_{run}_thermal_profile.png',bbox_inches='tight',dpi=500)

How long is the planetesimal below 700K?

In [ ]:
tacc = 0.8 #accretion time in Ma
rlen = int((r/1e3)*2)
tsint = np.zeros([rlen])
for i in range(rlen):
    tsint[i]=tall[Tall[i,:]>700][0]
tshort = np.min(tsint/Myr-tacc)
print(f'The shortest time before sintering is {tshort:.2f} Ma \
and the longest time before sintering is {np.max(tsint/Myr-tacc):.2f} Ma')

In [ ]:
rcheck = rplot[:-1] #remove surface point
rsint = rcheck[(tsint/Myr)>(tshort+tacc)]
print(f'The inner {rsint[0]}km sinters within {tshort:.1f} Ma of accretion')

How long is the planetesimal above rcmf?

In [ ]:
tacc = 0.8 #accretion time in Ma
rlen = int((r/1e3))
trcmf = np.zeros([rlen-1])
rcmf = params['rcmf'].values[0]
for i in range(rlen-1): #don't include surface
    trcmf[i]=tall[Tall[nmantle+i,:]>=(rcmf*(400)+1400)][-1]
max_rcmf = np.max(trcmf/Myr)
min_rcmf = np.min(trcmf/Myr)
#boundary layer thickness at rcmf
d0_rcmf1 = d0[t_plot_t<min_rcmf][0]
dl_rcmf1 = dl[t_plot_t<min_rcmf][-1]
d0_rcmf2 = d0[t_plot_t<min_rcmf][-1]
dl_rcmf2 = dl[t_plot_t<max_rcmf][-1]
print(f'The shortest time before T<T_C is {min_rcmf:.2f} Ma \
and the longest time before T<T_C is {max_rcmf:.2f} Ma')
print(f'The lid thickness at rcmf is {d0_rcmf1:.2f}-{d0_rcmf2:.2f}m and the CMB boundary layer thickness is {dl_rcmf1:.2f}-{dl_rcmf2:.2f}')

## Temperature and heat fluxes <a class="anchor" id="flux"></a>
<p align="right">(<a href="#top">back to top</a>)</p>

Have plotted only the central core temperature and convective mantle temperature.

This is Figure 7 from Sanderson et. al. 2024a.

In [ ]:
dl_end = int(dl[Tm_conv==0][0]/dr) #index of CMB b.l. base above CMB at cessation of convection

with sns.plotting_context('paper',font_scale=1.7,rc={'lines.linewidth':3}):
    fig, ax = plt.subplots(nrows=3,ncols=1,tight_layout=True,figsize=[15,15])
    xmin=tstart

    #temperatures as function of time
    ax[0].plot(t_plot_t,Tc,label='central core temperature',color='black')
    ax[0].plot(t_plot_t[Tm_conv!=0],Tm_conv[Tm_conv!=0],label='convective mantle interior temperature, $T_m$',color='#FF5350')
    ax[0].plot(t_plot_t[Tm_conv==0],T_profile[Tm_conv==0,nmantle+dl_end+2],label='conductive mantle temperature at final position of $\delta_l$',color='#FF5350',linestyle='dashed')
    ax[0].set(ylim=[1250,1550],ylabel='T/K')
    if log_time == True:
        ax[0].set_xscale('log')
    
    ax[0].legend(loc='lower left')
    ax[0].set_title('a) Temperatures',loc='left')
    #fluxes as function of time
    Fcmb_neg = Fcmb[Fcmb<0]
    Fcmb_pos = Fcmb[Fcmb>0]

    ln1, = ax[1].semilogy(t_plot_t,Fs,label='surface heat flux, $F_s$',color='#5F0F40',linestyle='dotted')
    ln2, = ax[1].plot(t_plot_t[Fcmb<0],abs(Fcmb_neg),label='negative CMB heat flux, $F_{CMB}<0$',color='#5BC0EB')
    ln3, = ax[1].plot(t_plot_t[Fcmb>0],Fcmb_pos,label='positive CMB heat flux, $F_{CMB}>0$',color='#0F4C5C')
    ln4, = ax[1].semilogy(t_plot_t,Fad,label='adiabatic heat flux $F_{ad}$',color='#E36414',linestyle='dashed')
    ln5, = ax[1].semilogy(t_plot_t,Frad,label='radiogenic heat flux $F_{rad}$',color='#9A031E',linestyle='-.')
    ln6 = ax[1].fill_betweenx(y=[1e-4,100],x1=diff_time,x2=tstrat_remove,alpha=0.35,color='#78c3fb',label='formation of stratification')
    ln7 = ax[1].fill_betweenx(y=[1e-4,100],x1=tstrat_remove,x2=terode,alpha=0.35,color='#8f2d56',label='erosion of stratification')
    ln8 = ax[1].fill_betweenx(y=[1e-4,100],x1=terode,x2=min_rcmf,alpha=0.35,color='#c7aa74',label='top of mantle above $\phi_C$')
    ln9 = ax[1].fill_betweenx(y=[1e-4,100],x1=min_rcmf,x2=max_rcmf,alpha=0.35,color='#957964',label='bottom of mantle above $\phi_C$')
    ln10 = ax[1].fill_betweenx(y=[1e-4,100],x1=fcond_t,x2=max(t_plot_t),alpha=0.35,color='#783f8e',label='mantle conducting')
    ln11 = ax[1].fill_betweenx(y=[1e-4,1e-3],x1=on1,x2=off1,alpha=0.35,color='grey',label='dynamo on')
    ax[1].fill_betweenx(y=[1e-4,1e-3],x1=on2,x2=off2,alpha=0.35,color='grey')
    if log_time == True:
        ax[1].set_xscale('log')
    ax[1].set(ylim=[1e-4,1e2],ylabel='Heat flux / W$m^{-2}$')
    #ax[1].legend(bbox_to_anchor=[1,0.9],ncol=1)
    leg1 = ax[1].legend(handles=[ln1,ln2,ln3,ln4,ln5],bbox_to_anchor=(0.7,1))
    leg2 = ax[1].legend(handles=[ln6,ln7,ln8,ln9,ln10,ln11],bbox_to_anchor=(1,1))
    ax[1].add_artist(leg1)
    ax[1].add_artist(leg2)
    ax[1].set_title('b) Mantle',loc='left')
    
    #core power
    wn = 200 #int(len(t_plot_t[t_plot_t>tsolid_start])/20)
    ax[2].plot(t_plot_t,Fcmb*4*np.pi*rc**2/1e9,label='CMB heat flow, $Q_{CMB}$',color='#0F4C5C')
    ax[2].plot(t_plot_t,qrad,label='radiogenic heat, $Q_R$',color='#9A031E',linestyle='-.')
    ax[2].plot(t_plot_t[t_plot_t<tsolid_start],qs[t_plot_t<tsolid_start],label='secular cooling, $Q_S$',color='#E36414',linestyle='dashed')
    ax[2].plot(t_plot_t[t_plot_t>tsolid_start],pd.Series(qs[t_plot_t>tsolid_start]).rolling(window=wn).mean(),color='#E36414',linestyle='dashed')
    ax[2].plot(t_plot_t[t_plot_t>tsolid_start],pd.Series(ql[t_plot_t>tsolid_start]).rolling(window=wn).mean(),label='latent heat, $Q_L$',color='#5BC0EB',linestyle='dotted')
    #ax[2].plot(t_plot_t[t_plot_t>tsolid_start],pd.Series(qg[t_plot_t>tsolid_start]).rolling(window=wn).mean(),label='gravitational energy,$Q_G$')
    ax[2].legend(loc='upper right')
    ax[2].set_title('c) Core',loc='left')
    if log_time == True:
        ax[2].set_xscale('log')
    ax[2].set(ylabel='Power/GW',xlabel='Time /Ma after CAI formation')

    if save == True:
        plt.savefig(f'{save_path}run_{run}_flux.png',dpi=500)

## Magnetic Field Strength <a class="anchor" id="Bfield"></a>
<p align="right">(<a href="#top">back to top</a>)</p>

Extract compositional and thermal buoyancy components

In [ ]:
comp = buoyr[0,:] #compostional buoyancy component
therm = buoyr[1,:] #thermal buoyancy component

In [ ]:
Xs_eutectic = 33
print(f'The core reaches the eutectic at {t_plot_t[Xs>=Xs_eutectic][0]:.2f} Ma')

Create data frames for rolling average compositional dynamo strengths

In [ ]:
compdf = pd.Series(comp[f<f0])
thermdf = pd.Series(therm[f<f0])
Remdf = pd.Series(Rem[f<f0])
Bdf = pd.Series(B[f<f0])

Make figure - Figure 8 in Sanderson et. al. 2024b

There is a lag between the onset of solidification and the rolling average so plot the original time series and the average.

In [ ]:
wn = 75 #averaging window width
fig, ax = plt.subplots(nrows=2,ncols=1,sharex='col',figsize=[10,8])
#B and Rem
ax2 = ax[0].twinx()
ln1 = ax[0].plot(t_plot_t[f>=f0],np.ma.masked_where(Rem[f>=f0]<10,B[f>=f0]),color='black')
ax[0].plot(t_plot_t[f<f0],np.ma.masked_where(Rem[f<f0]<10,B[f<f0]),color='black',alpha=0.1) #original series
ax[0].plot(t_plot_t[f<f0],Bdf.rolling(window=wn,center=True).mean(),color='black') #rolling average
ln2 = ax2.plot(t_plot_t[f>=f0],Rem[f>=f0],color='darkorchid')
ax2.plot(t_plot_t[f<f0],Rem[f<f0],color='darkorchid',alpha=0.1) #original series
ax2.plot(t_plot_t[f<f0],Remdf.rolling(window=wn,center=True).mean(),color='darkorchid') #rolling average


ax2.hlines(10,min(t_plot_t),max(t_plot_t),linestyle='dashed',color='darkorchid')
ax[0].legend(ln1+ln2,['Magnetic field strength','Re$_m$'])
ax[0].set_ylabel('Magnetic field strength/ $\mu T$')
ax2.set_ylabel('$Re_m$')
ax[0].tick_params(axis='y',colors='black')
ax2.tick_params(axis='y',colors='darkorchid')
ax[0].yaxis.label.set_color('black') 
ax2.yaxis.label.set_color('darkorchid') 

#thermal and compositional buoyancy flux
ln3 = ax[1].plot(t_plot_t[f>=f0],therm[f>=f0],color='black')
#ax[1].plot(t_plot_t[f<f0],therm[f<f0],color='black',alpha=0.1)
ax[1].plot(t_plot_t[f<f0],thermdf.rolling(window=wn,center=True).mean(),color='black')
ax[1].plot(t_plot_t[f<f0],comp[f<f0],color='darkorchid',alpha=0.1)
ln4=ax[1].plot(t_plot_t[f<f0],compdf.rolling(window=wn,center=True).mean(),color='darkorchid')
ax[1].legend(ln3+ln4,['thermal','compositional'])
ax[1].set(ylim=[1e-11,1e-8],ylabel='Buoyancy flux per unit area /kg$s^{-1}m^{-2}$',xlabel='Time/Ma',xscale='log')

if save == True:
    plt.savefig(f'{save_path}run_{run}_Bbuoy.png',dpi=500)